In [ ]:
encoder_url="https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"
preprocess_url="https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

In [ ]:
!pip install --upgrade tensorflow_hub
import tensorflow as tf
import tensorflow_hub as hub


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#!pip3 install tensorflow_text
!python -m pip install tensorflow_text
import tensorflow_text as text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
##load the dataset
import io
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [ ]:
df = pd.read_csv(io.BytesIO(uploaded.get('train.csv')))
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'],df['target'])


In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
#Intialize bert layer this convert text into vector
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

In [ ]:
#Intialize the neural network

#This layer will be used to prevent model overfitting
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
#Dense layer only has one neuron. We also initialize the activation function as sigmoid. sigmoid is used when we have output values that between 0 and 1.
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

In [ ]:
#We also name the layer as output because this is our output layer. Lets now add the input and output layers to construct the final model as shown below 
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_2 (KerasLayer)     {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/10
179/179 [==============================] - 2549s 14s/step - loss: 0.6294 - accuracy: 0.6444 - precision: 0.6034 - recall: 0.4881
Epoch 2/10
179/179 [==============================] - 2524s 14s/step - loss: 0.5959 - accuracy: 0.6872 - precision: 0.6536 - recall: 0.5689
Epoch 3/10
179/179 [==============================] - 2596s 14s/step - loss: 0.5836 - accuracy: 0.7071 - precision: 0.6760 - recall: 0.6034
Epoch 4/10
179/179 [==============================] - 2593s 14s/step - loss: 0.5731 - accuracy: 0.7087 - precision: 0.6825 - recall: 0.5943
Epoch 5/10
179/179 [==============================] - 2532s 14s/step - loss: 0.5645 - accuracy: 0.7232 - precision: 0.6999 - recall: 0.6161
Epoch 6/10
113/179 [=================>............] - ETA: 15:31 - loss: 0.5591 - accuracy: 0.7235 - precision: 0.7010 - recall: 0.6187